In [2]:
#Install Java Runtime 8
import os       #importing os to set environment variable
!apt-get remove -y java*
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'java-propose-classpath' for glob 'java*'
Note, selecting 'java2-sdk' for glob 'java*'
Note, selecting 'java2html' for glob 'java*'
Note, selecting 'java9-sdk-headless' for glob 'java*'
Note, selecting 'java-gcj-compat' for glob 'java*'
Note, selecting 'javascript-common' for glob 'java*'
Note, selecting 'java11-runtime' for glob 'java*'
Note, selecting 'javacc4' for glob 'java*'
Note, selecting 'java3ds-fileloader' for glob 'java*'
Note, selecting 'java10-runtime' for glob 'java*'
Note, selecting 'javahelp2-doc' for glob 'java*'
Note, selecting 'java-sdk-headless' for glob 'java*'
Note, selecting 'java-sdk' for glob 'java*'
Note, selecting 'java6-sdk-headless' for glob 'java*'
Note, selecting 'java11-sdk' for glob 'java*'
Note, selecting 'java10-sdk-headless' for glob 'java*'
Note, selecting 'javalex' for glob 'java*'
Note, selecting 'java-compiler' for glob 'java*'
Note, se

In [3]:
#Initialize Spark Runtime Environment
!export JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64/jre
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import SQLContext

spark = SparkSession.builder \
   .master("local") \
   .appName("Wordcount & Friends Tutorial") \
   .config("spark.executor.memory", "2gb") \
   .getOrCreate()
   
sc = spark.sparkContext
sqlContext = SQLContext(sc)

sc.version

     |████████████████████████████████| 204.2MB 78kB/s 
     |████████████████████████████████| 204kB 53.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=2e741d76b79d06e400b5078d9d31aafc09bad1977e5560ad3239d32655cfb110
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


'3.0.1'

PART-2: "COMMON FRIENDS" EXAMPLE USINg MAPREDUCE

In [4]:
# Load data from github
!wget "https://raw.githubusercontent.com/ibarabasi/wordcount/master/friends"
rdd = sc.textFile("friends")
!cat friends

--2020-11-30 21:28:34--  https://raw.githubusercontent.com/ibarabasi/wordcount/master/friends
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 193 [text/plain]
Saving to: ‘friends’

friends             100%[===================>]     193  --.-KB/s    in 0s      

2020-11-30 21:28:34 (12.3 MB/s) - ‘friends’ saved [193/193]

me Alice
Henry me
Henry Alice
me Jane
Alice John
Jane John
Judy Alice
me Mary
Mary Joyce
Joyce Henry
Judy me
Judy Jane
John Carol
Carol me
Mary Henry
Louise Ronald
Ronald Thomas
William Thomas


In [5]:

# Print the RDD content
rdd.take(20)

['me Alice',
 'Henry me',
 'Henry Alice',
 'me Jane',
 'Alice John',
 'Jane John',
 'Judy Alice',
 'me Mary',
 'Mary Joyce',
 'Joyce Henry',
 'Judy me',
 'Judy Jane',
 'John Carol',
 'Carol me',
 'Mary Henry',
 'Louise Ronald',
 'Ronald Thomas',
 'William Thomas']

In [10]:
word_counts = rdd.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: (a + b)).map(lambda x:(x[1],x[0]))


word_counts.take(30)


[(6, 'me'),
 (4, 'Alice'),
 (4, 'Henry'),
 (3, 'Jane'),
 (3, 'John'),
 (3, 'Judy'),
 (3, 'Mary'),
 (2, 'Joyce'),
 (2, 'Carol'),
 (1, 'Louise'),
 (2, 'Ronald'),
 (2, 'Thomas'),
 (1, 'William')]

In [11]:
rdd1=rdd.map(lambda x: x.split()).union(rdd.map(lambda x: x.split()[::-1]))
# Bring my friend list to local
lst = rdd1.filter(lambda x: x[0] == 'me').map(lambda x: x[1]).collect()
# Build the second pair RDD
rdd2 = rdd1.filter(lambda x: x[0] in lst).map(lambda x: x[1]). \
    filter(lambda x: x != 'me' and x not in lst). \
    map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b). \
    map(lambda x: (x[1], x[0])).sortByKey(ascending = False)
# Bring the result to local since the sample is small
for x, y in rdd2.collect():
   print ("The stranger {} has {} common friends with me".format(y, x))

The stranger John has 3 common friends with me
The stranger Joyce has 2 common friends with me
